<a href="https://colab.research.google.com/github/hyojunyee/kita_2404/blob/main/m8_VISION%20%EC%9D%91%EC%9A%A9/vision_07_object_detection_YOLOv5_video_1002.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


https://www.youtube.com/watch?v=NcaGFp76BTY

import shutil as sh
- shutil: 파일과 디렉터리를 복사, 이동, 삭제하는 등의 작업을 수행하는 라이브러리입니다.
- 예시: 파일 복사, 디렉터리 이동, 파일 삭제 등.
코드에서 파일 관리 작업을 보다 쉽게 처리하기 위해 사용될 수 있습니다.

from base64 import b64encode
- base64.b64encode: 데이터를 Base64로 인코딩하는 기능을 제공합니다. 주로 이진 데이터를 텍스트로 변환하는 데 사용됩니다.
- 예시: 이미지를 Base64 문자열로 인코딩하여 웹에서 전송하거나, HTML 내에 이미지를 인라인으로 포함시킬 때 사용됩니다.

In [2]:
import os, time, random, cv2, torch
import numpy as np
import pandas as pd
import shutil as sh
from tqdm.auto import tqdm

# from IPython.display import Video, HTML   # colab에서 사용 불가능
from base64 import b64encode    # 이진 파일(ex: 비디오 파일)을 텍스트 형식으로 변환하여 HTML 문서나 소스 코드 내에 직접 포함시킬 수 있게 해줌

In [15]:
%%time

!git clone http://github.com/ultralytics/yolov5
%cd yolov5
!pip install -U pycocotools
!pip install -r requirements.txt
!cp yolo5/requirements.txt ./

fatal: destination path 'yolov5' already exists and is not an empty directory.
/content/yolov5
cp: cannot stat 'yolo5/requirements.txt': No such file or directory
CPU times: user 69.7 ms, sys: 13 ms, total: 82.7 ms
Wall time: 6.15 s


Google Colab에서 유튜브 동영상을 다운로드 받기 위해 yt-dlp 라이브러리를 사용

In [4]:
!pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.2/171.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.1/164.1 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.2 MB/s eta 0:00:00


In [5]:
import yt_dlp

# 유튜브 비디오 URL
rul = 'https://www.youtube.com/watch?v=NcaGFp76BTY'

# yt-dlp를 사용하여 비디오 다운로드
ydl_opts = {'outtmpl': 'downloaded_video.mp4'}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([rul])

[youtube] Extracting URL: https://www.youtube.com/watch?v=NcaGFp76BTY
[youtube] NcaGFp76BTY: Downloading webpage
[youtube] NcaGFp76BTY: Downloading ios player API JSON
[youtube] NcaGFp76BTY: Downloading web creator player API JSON
[youtube] NcaGFp76BTY: Downloading player d9418494
[youtube] NcaGFp76BTY: Downloading m3u8 information
[info] NcaGFp76BTY: Downloading 1 format(s): 136+251
[download] Destination: downloaded_video.mp4.f136.mp4
[download] 100% of    7.52MiB in 00:00:00 at 19.71MiB/s  
[download] Destination: downloaded_video.mp4.f251.webm
[download] 100% of   27.29KiB in 00:00:00 at 289.91KiB/s 
[Merger] Merging formats into "downloaded_video.mp4.mkv"
Deleting original file downloaded_video.mp4.f251.webm (pass -k to keep)
Deleting original file downloaded_video.mp4.f136.mp4 (pass -k to keep)


In [6]:
!ls

downloaded_video.mp4.mkv  drive  sample_data  yolov5


In [7]:
# ffmpeg는 멀티미디어 파일(동영상, 오디오 파일 등)을 처리하는 데 사용, 다양한 포맷으로 변환하거나, 오디오/비디오를 편집, 인코딩, 딩코딩 할 수 있음
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [8]:
# -c는 사용할 코덱을 지정하는 옵션, 여기서 copy는 코덱을 지정하지 않고, 기존 인코딩을 그대로 복사하라는 의미
!ffmpeg -i downloaded_video.mp4.mkv -c copy downloaded_video.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [9]:
!ls

downloaded_video.mp4  downloaded_video.mp4.mkv	drive  sample_data  yolov5


In [14]:
!python /content/yolov5/detect.py  --source downloaded_video.mp4 --weights yolov5s.pt --conf 0.25

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
detect: weights=['yolov5s.pt'], source=downloaded_video.mp4, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
requirements: Ultralytics requirement ['gitpython>=3.1.30'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 28.7 MB/s eta 0:00:00
   ━━━━━━━━

In [26]:
%cd /content

/content


In [27]:
!ls

downloaded_video.mp4  downloaded_video.mp4.mkv	drive  sample_data  yolov5  yolov5s.pt


In [28]:
!pwd

/content


In [29]:
import os
filenames = ''
for (filenames) in os.walk("/content/yolov5/runs/detect/exp"):
    filenames = filenames
    print(filenames)


('/content/yolov5/runs/detect/exp', [], ['downloaded_video.mp4'])


In [32]:
!python yolov5.py --source downloaded_video.mp4 --weghts yolov5s.pt --conf 0.25

python3: can't open file '/content/yolov5.py': [Errno 2] No such file or directory


In [31]:
from google.colab import files

# 예 코렙에서 생성한 'output.txt'파일을 로컬로 다운로드
files.download('/content/yolov5/runs/detect/exp/downloaded_video.mp4')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

shutil을 이용한 파일복사
https://zephyrus1111.tistory.com/199

In [33]:
import shutil
shutil.copy('/content/yolov5/runs/detect/exp/downloaded_video.mp4', '/content/drive/MyDrive/kdt_240424/m8_vision/data/output_video.mp4')

'/content/drive/MyDrive/kdt_240424/m8_vision/data/output_video.mp4'